<h1>拉普拉斯方法定小行星轨道<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#拉普拉斯方法定轨道六根数" data-toc-modified-id="拉普拉斯方法定轨道六根数-0"><span class="toc-item-num">0&nbsp;&nbsp;</span>拉普拉斯方法定轨道六根数</a></span></li><li><span><a href="#已知三次观测的观测坐标，定出某一时刻的位置矢量和速度矢量" data-toc-modified-id="已知三次观测的观测坐标，定出某一时刻的位置矢量和速度矢量-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>已知三次观测的观测坐标，定出某一时刻的位置矢量和速度矢量</a></span><ul class="toc-item"><li><span><a href="#PART-1-星表的format" data-toc-modified-id="PART-1-星表的format-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>PART 1 星表的format</a></span><ul class="toc-item"><li><span><a href="#给定天文常数，读取原始星表" data-toc-modified-id="给定天文常数，读取原始星表-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>给定天文常数，读取原始星表</a></span></li><li><span><a href="#接下来：将所有观测资料归到太阳系坐标系" data-toc-modified-id="接下来：将所有观测资料归到太阳系坐标系-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>接下来：将所有观测资料归到太阳系坐标系</a></span></li><li><span><a href="#缺陷：" data-toc-modified-id="缺陷：-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>缺陷：</a></span></li><li><span><a href="#或者可以尝试使用反算星历表的方式获取太阳坐标？？" data-toc-modified-id="或者可以尝试使用反算星历表的方式获取太阳坐标？？-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>或者可以尝试使用反算星历表的方式获取太阳坐标？？</a></span></li><li><span><a href="#插值获得地球相对于日心的坐标。" data-toc-modified-id="插值获得地球相对于日心的坐标。-1.1.5"><span class="toc-item-num">1.1.5&nbsp;&nbsp;</span>插值获得地球相对于日心的坐标。</a></span></li><li><span><a href="#求出测站相对于太阳的坐标，求出Pj，Qj" data-toc-modified-id="求出测站相对于太阳的坐标，求出Pj，Qj-1.1.6"><span class="toc-item-num">1.1.6&nbsp;&nbsp;</span>求出测站相对于太阳的坐标，求出Pj，Qj</a></span></li></ul></li><li><span><a href="#PART-2-给出t0,算出t0的F,G,进而得出r0,v0" data-toc-modified-id="PART-2-给出t0,算出t0的F,G,进而得出r0,v0-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>PART 2 给出t0,算出t0的F,G,进而得出r0,v0</a></span></li></ul></li><li><span><a href="#已知某一时刻的位置矢量和速度矢量，求轨道六根数" data-toc-modified-id="已知某一时刻的位置矢量和速度矢量，求轨道六根数-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>已知某一时刻的位置矢量和速度矢量，求轨道六根数</a></span></li><li><span><a href="#由轨道6根数归算星历表" data-toc-modified-id="由轨道6根数归算星历表-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>由轨道6根数归算星历表</a></span></li><li><span><a href="#Warnning:-P，Q反算出来的值不对" data-toc-modified-id="Warnning:-P，Q反算出来的值不对-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Warnning: P，Q反算出来的值不对</a></span></li><li><span><a href="#计算残差-见-step3---echo1st-(在小行星部分不计算）" data-toc-modified-id="计算残差-见-step3---echo1st-(在小行星部分不计算）-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>计算残差 见 step3 - echo1st (在小行星部分不计算）</a></span></li></ul></div>

# 拉普拉斯方法定轨道六根数

# 已知三次观测的观测坐标，定出某一时刻的位置矢量和速度矢量
按照笔记的方法分别推出各个量

注意时间单位的选取!

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import datetime
import time

from scipy import interpolate 

from DDS_Laplace import *

import DDS_CMpack as DDS

%matplotlib inline

## PART 1 星表的format

### 给定天文常数，读取原始星表

In [2]:
# astronomy constants
R_earth = 6371 #km
au = 149597870 #km
r_station_earth = 0.999102 #*R_earth
r_earth_sun = 1*au

### unit for time -> artifact satellite
# time_unit = 806.81163 #806.8116 # SI 

## unit for time -> planet
time_unit_day = 58.1324409 # Mean solar day
time_unit = time_unit_day * 86400

J2000 = 2451545.0 # days

In [3]:
## 我们需要先获得在观测时间内（1998年11月25日 至 12月6日）的太阳位置——需要插值

# from tdt(tt) to utc: utc = tdt(tt) + delta_tt2utc

delta_tdt2utc = -0.0039 - 32.184 # SI

# tt = tai + 32.184
# ut = tai - 0.0039 => tai = ut + 0.0039
# ut = tt - 32.184 - 0.0039 # SI

In [4]:
# give observed position
year = 1998

Asteriod_fram = pd.read_csv('Asteroid.csv')

Asteriod_fram


,Number,lambda,Rcos_fai,Rsin_fai,month,day,alpha,delta
0,97,34.7625,0.86165,0.50608,12,4.13532,10.960203,19.700000
1,97,34.7625,0.86165,0.50608,12,4.14534,10.960672,19.700000
2,97,34.7625,0.86165,0.50608,12,5.09543,11.006331,19.783333
3,97,34.7625,0.86165,0.50608,12,5.09868,11.006483,19.783333
4,758,279.2379,0.88044,0.47257,12,3.36964,10.923786,19.625083
5,758,279.2379,0.88044,0.47257,12,3.37277,10.923942,19.625583
6,758,279.2379,0.88044,0.47257,12,3.38338,10.924431,19.726667
7,758,279.2379,0.88044,0.47257,12,3.38649,10.924575,19.627000
8,758,279.2379,0.88044,0.47257,12,3.39707,10.925042,19.627806
9,758,279.2379,0.88044,0.47257,12,4.37163,10.971636,19.724611


In [5]:
# give sun's position (in tdt = 0)

Sun_fram = pd.read_csv('Sun.csv')

Sun_fram

,month,day,x,y,z
0,11,25,-0.455432,-0.803603,-0.348406
1,11,26,-0.439814,-0.810697,-0.351482
2,11,27,-0.424063,-0.817540,-0.354450
3,11,28,-0.408182,-0.824132,-0.357309
4,11,29,-0.392177,-0.830469,-0.360057
5,11,30,-0.376053,-0.836550,-0.362694
6,12,1,-0.359815,-0.842373,-0.365219
7,12,2,-0.343468,-0.847938,-0.367632
8,12,3,-0.327015,-0.853242,-0.369931
9,12,4,-0.310463,-0.858284,-0.372117


### 接下来：将所有观测资料归到太阳系坐标系

需要将所有观测资料归到以utc 为时间单位的 相对于太阳的 太阳系坐标系
> 目前的太阳：（太阳-地球坐标）
> 1. 转换为地球-太阳坐标 R_earth_sun = -R_sun_earth
> 2. 时间单位需要转化

> 目前的小行星：测站坐标
> 1. 转换到地心坐标 R_earth = R_station + R_asteroid_station
> 2. 转换到太阳坐标 R_asteroid_sun = R_earth + R_earth_sun

### 缺陷：
1. 没有使用太阳系质心力学时
2. 我们归算的质心有问题（它并不是绕太阳旋转的——而应该是太阳系质心）


In [6]:
# add JD and SG to Asteriod_fram
year = 1998

SG_t = []; SG_d_t = []; jd_t = []; jd_s_t = []

for i in range(len(Asteriod_fram)):
    month = Asteriod_fram.loc[i,'month']
    day_hour = Asteriod_fram.loc[i,'day']
    day_int = int(day_hour)
    hours = (day_hour - day_int)/24
    SG,jd = DDS.UTC2SG(year,month,day_int,hours)
    
    SG_t.append(SG); jd_t.append(jd)
    SG_d_t.append(SG%(3600*24)/(3600*24)*360)
    jd_s_t.append( (jd)*24*3600 )
    
Asteriod_fram_right = pd.DataFrame({'SG':np.array(SG_t),'SG_d':np.array(SG_d_t),
                                    'jd':np.array(jd_t),'jd_s':np.array(jd_s_t)})

Asteriod_fram_fil = pd.concat([Asteriod_fram,Asteriod_fram_right],axis=1)

Asteriod_fram_fil

,Number,lambda,Rcos_fai,Rsin_fai,month,day,alpha,delta,SG,SG_d,jd,jd_s
0,97,34.7625,0.86165,0.50608,12,4.13532,10.960203,19.700000,-3.402409e+07,72.961353,-393.499022,-3.399832e+07
1,97,34.7625,0.86165,0.50608,12,4.14534,10.960672,19.700000,-3.402409e+07,72.967632,-393.499005,-3.399831e+07
2,97,34.7625,0.86165,0.50608,12,5.09543,11.006331,19.783333,-3.393746e+07,73.922000,-392.499091,-3.391192e+07
3,97,34.7625,0.86165,0.50608,12,5.09868,11.006483,19.783333,-3.393746e+07,73.924037,-392.499086,-3.391192e+07
4,758,279.2379,0.88044,0.47257,12,3.36964,10.923786,19.625083,-3.411069e+07,72.122556,-394.498615,-3.408468e+07
5,758,279.2379,0.88044,0.47257,12,3.37277,10.923942,19.625583,-3.411069e+07,72.124518,-394.498610,-3.408468e+07
6,758,279.2379,0.88044,0.47257,12,3.38338,10.924431,19.726667,-3.411069e+07,72.131167,-394.498592,-3.408468e+07
7,758,279.2379,0.88044,0.47257,12,3.38649,10.924575,19.627000,-3.411069e+07,72.133116,-394.498586,-3.408468e+07
8,758,279.2379,0.88044,0.47257,12,3.39707,10.925042,19.627806,-3.411069e+07,72.139747,-394.498568,-3.408468e+07
9,758,279.2379,0.88044,0.47257,12,4.37163,10.971636,19.724611,-3.402405e+07,73.109451,-393.498612,-3.399828e+07


In [7]:
# R_earth_sun
Earth_sun_fram = Sun_fram.loc[:,['month','day']]
Earth_sun_fram[['x','y','z']] = Sun_fram.loc[:,['x','y','z']]*(-1)

Earth_sun_fram['hours'] = np.ones(len(Earth_sun_fram))*delta_tdt2utc/(24*3600)

# utc,au to utc,jd,au
year = 1998
SG_total = [];jd_total = [];SG_d_total = [];jd_s_total = [];
for i in range(len(Earth_sun_fram)):
    SG,jd = DDS.UTC2SG(year,Earth_sun_fram.loc[i,'month'],\
                       Earth_sun_fram.loc[i,'day'],Earth_sun_fram.loc[i,'hours'])
    SG_total.append(SG)
    SG_d_total.append(SG%(3600*24)/(3600*24)*360)
    jd_total.append(jd)
    jd_s_total.append( (jd)*24*3600 )

Earth_sun_fram_right = pd.DataFrame({'SG':np.array(SG_total),
                                     'SG_d':np.array(SG_d_total),
                                     'jd':np.array(jd_total),
                                    'jd_s':np.array(jd_s_total)})

Earth_sun_fram_fil = pd.concat([Earth_sun_fram,Earth_sun_fram_right],axis=1)

Earth_sun_fram_fil

,month,day,x,y,z,hours,SG,SG_d,jd,jd_s
0,11,25,0.455432,0.803603,0.348406,-0.000373,-3.480384e+07,64.000116,-402.499273,-3.477594e+07
1,11,26,0.439814,0.810697,0.351482,-0.000373,-3.471720e+07,64.985764,-401.499273,-3.468954e+07
2,11,27,0.424063,0.817540,0.354450,-0.000373,-3.463057e+07,65.971411,-400.499273,-3.460314e+07
3,11,28,0.408182,0.824132,0.357309,-0.000373,-3.454393e+07,66.957058,-399.499273,-3.451674e+07
4,11,29,0.392177,0.830469,0.360057,-0.000373,-3.445729e+07,67.942706,-398.499273,-3.443034e+07
5,11,30,0.376053,0.836550,0.362694,-0.000373,-3.437066e+07,68.928353,-397.499273,-3.434394e+07
6,12,1,0.359815,0.842373,0.365219,-0.000373,-3.428402e+07,69.914000,-396.499273,-3.425754e+07
7,12,2,0.343468,0.847938,0.367632,-0.000373,-3.419738e+07,70.899648,-395.499273,-3.417114e+07
8,12,3,0.327015,0.853242,0.369931,-0.000373,-3.411075e+07,71.885295,-394.499273,-3.408474e+07
9,12,4,0.310463,0.858284,0.372117,-0.000373,-3.402411e+07,72.870943,-393.499273,-3.399834e+07


### 或者可以尝试使用反算星历表的方式获取太阳坐标？？

* 用地球相对于太阳的位置去算地球的轨道根数，进而反算星历表

### 插值获得地球相对于日心的坐标。

* 以jd_s (秒为单位的儒略日）作为插值的自变量，x,y,z三个方向单独拟合
* 或者 取 $$ SG = 18.6973746*3600 + 879000.0513367*3600*t + 0.093104*t**2 - 6.2e-6*t**3 $$
    这个公式直接算出来的SG作为插值的x节点（这样,用 SG%(3600*24)/(3600*24)*360 即可求得我们需要的当天的恒星时）
    
* 目前取第一种方案做

In [8]:
# 插值获得所有太阳坐标

# from scipy import interpolate 

# x,y,z三个方向分别独立插值 -- 因为运动的独立性，所以可以直接分开用样条插值

axis = ['x','y','z']
xyz_bspline = []
for ax in axis:
    y = Earth_sun_fram_fil[ax]
    x = Earth_sun_fram_fil['jd_s'].values
    
    x_new = Asteriod_fram_fil['jd_s'].values
    
    f_linear = interpolate.interp1d(x, y)

    tck = interpolate.splrep(x, y)

    y_bspline = interpolate.splev(x_new, tck) # 插值得到的太阳相对于地球的xyz坐标
    
    xyz_bspline.append(y_bspline)

XYZ_fram = pd.DataFrame({'X_e_s':xyz_bspline[0],
                       'Y_e_s':xyz_bspline[1],
                       'Z_e_s':xyz_bspline[2]})


Earth_sun_fram_bspline = pd.concat([XYZ_fram,Asteriod_fram_fil[['jd_s','jd']]],axis=1)
Earth_sun_fram_bspline

,X_e_s,Y_e_s,Z_e_s,jd_s,jd
0,0.310459,0.858285,0.372118,-3.399832e+07,-393.499022
1,0.310458,0.858286,0.372118,-3.399831e+07,-393.499005
2,0.293812,0.863064,0.374189,-3.391192e+07,-392.499091
3,0.293812,0.863064,0.374189,-3.391192e+07,-392.499086
4,0.327004,0.853245,0.369933,-3.408468e+07,-394.498615
5,0.327004,0.853245,0.369933,-3.408468e+07,-394.498610
6,0.327004,0.853246,0.369933,-3.408468e+07,-394.498592
7,0.327004,0.853246,0.369933,-3.408468e+07,-394.498586
8,0.327004,0.853246,0.369933,-3.408468e+07,-394.498568
9,0.310452,0.858287,0.372119,-3.399828e+07,-393.498612


### 求出测站相对于太阳的坐标，求出Pj，Qj

** 我们已经有的表：**

1. Earth_sun_fram_bspline: 
    * 包含地球相对于日心的位置，
    * 以有观测数据的jd_s为标注，
    * 插值节点

2. Asteriod_fram_fil
    * 观测数据

In [9]:
# 求出我们所需要的相对于太阳的坐标：

# 现求出测站相对于太阳的坐标(au为单位)
station_sun = []
for i in range(len(Asteriod_fram_fil)):
    # 需要根据alpha和delta求出所需的测站到地心的RA(u),DEC(v)-不用-(星表中自带了R*sin/cos phi')
    
    RA_station_earth = Asteriod_fram_fil.loc[i,'SG_d'] + Asteriod_fram_fil.loc[i,'lambda']
    u = RA_station_earth*2*np.pi/360
    
    # 加号前面的是测站到地心的矢量，加上地心到日心的矢量，即可求的测站到日心的矢量
    
    x = np.cos(u)*Asteriod_fram_fil.loc[i,'Rcos_fai']*R_earth/au + Earth_sun_fram_bspline.loc[i,'X_e_s']
    y = np.sin(u)*Asteriod_fram_fil.loc[i,'Rcos_fai']*R_earth/au + Earth_sun_fram_bspline.loc[i,'Y_e_s']
    z = Asteriod_fram_fil.loc[i,'Rsin_fai']*R_earth/au + Earth_sun_fram_bspline.loc[i,'Z_e_s']
    
    station_sun.append([x,y,z])
    
    
station_sun_fram = pd.DataFrame(np.array(station_sun),columns=['X','Y','Z'])

In [10]:
## 给出最后我们需要的表，此时还应该包含P,Q

position_fram = pd.concat([Asteriod_fram_fil,station_sun_fram],axis=1)

# add Pj ,Qj to positon_fram_fil
Lt = []; Pt = []; Qt = []
Lambda = []; Miu = []; Niu = [];

#由 delta,alpha 算出 L = [Lambda,Miu,Niu]
for j in range(len(position_fram)):
    delta = position_fram['delta'].values[j]*2*np.pi/360 ; 
    alpha = position_fram['alpha'].values[j]*2*np.pi/360;

    L = [np.cos(delta)*np.cos(alpha),\
         np.cos(delta)*np.sin(alpha),\
         np.sin(delta)]
    
    Lambda.append(L[0]); 
    Miu.append(L[1]);
    Niu.append(L[2]);
    
#     Lambda.append(np.cos(delta)*np.cos(alpha))
#     Miu.append(np.cos(delta)*np.sin(alpha))
#     Niu.append(np.sin(delta))

# 由[Lambda,Miu,Niu] 和 测站-地心矢量归算出 P,Q
    Pj = L[2]*station_sun[j][0] - L[0]*station_sun[j][2]
    Qj = L[2]*station_sun[j][1] - L[1]*station_sun[j][2]
    
    Lt.append(L)
    Pt.append(Pj)
    Qt.append(Qj)
#     print(Lt,Pt,Qt)

position_fram_right = pd.DataFrame({'P':np.array(Pt),'Q':np.array(Qt),
                                    'Lambda':np.array(Lambda),
                                    'Miu':np.array(Miu),'Niu':np.array(Niu)})

position_fram_fil = pd.concat([position_fram,position_fram_right],axis=1)

In [11]:
## 将相对于J2000的时间序列jd_s（unit = s) 单位取做理论单位

TinUnit = (position_fram_fil['jd_s'])/time_unit
position_fram_fil['formatT'] = TinUnit

In [12]:
position_fram_fil.to_csv('Asteriod_format.csv')
position_fram_fil

,Number,lambda,Rcos_fai,Rsin_fai,month,day,alpha,delta,SG,SG_d,...,jd_s,X,Y,Z,Lambda,Miu,Niu,P,Q,formatT
0,97,34.7625,0.86165,0.50608,12,4.13532,10.960203,19.700000,-3.402409e+07,72.961353,...,-3.399832e+07,0.310448,0.858320,0.372139,0.924298,0.178999,0.337095,-0.239317,0.222723,-6.769009
1,97,34.7625,0.86165,0.50608,12,4.14534,10.960672,19.700000,-3.402409e+07,72.967632,...,-3.399831e+07,0.310447,0.858320,0.372139,0.924296,0.179007,0.337095,-0.239317,0.222720,-6.769009
2,97,34.7625,0.86165,0.50608,12,5.09543,11.006331,19.783333,-3.393746e+07,73.922000,...,-3.391192e+07,0.293800,0.863099,0.374211,0.923671,0.179649,0.338464,-0.246207,0.224901,-6.751808
3,97,34.7625,0.86165,0.50608,12,5.09868,11.006483,19.783333,-3.393746e+07,73.924037,...,-3.391192e+07,0.293800,0.863099,0.374211,0.923671,0.179652,0.338464,-0.246207,0.224900,-6.751808
4,758,279.2379,0.88044,0.47257,12,3.36964,10.923786,19.625083,-3.411069e+07,72.122556,...,-3.408468e+07,0.327042,0.853240,0.369953,0.924843,0.178495,0.335864,-0.232307,0.220538,-6.786204
5,758,279.2379,0.88044,0.47257,12,3.37277,10.923942,19.625583,-3.411069e+07,72.124518,...,-3.408468e+07,0.327041,0.853240,0.369953,0.924840,0.178497,0.335872,-0.232303,0.220544,-6.786204
6,758,279.2379,0.88044,0.47257,12,3.38338,10.924431,19.726667,-3.411069e+07,72.131167,...,-3.408468e+07,0.327041,0.853240,0.369953,0.924255,0.178392,0.337533,-0.231544,0.222000,-6.786204
7,758,279.2379,0.88044,0.47257,12,3.38649,10.924575,19.627000,-3.411069e+07,72.133116,...,-3.408468e+07,0.327041,0.853240,0.369953,0.924830,0.178506,0.335895,-0.232292,0.220561,-6.786204
8,758,279.2379,0.88044,0.47257,12,3.39707,10.925042,19.627806,-3.411069e+07,72.139747,...,-3.408468e+07,0.327041,0.853240,0.369953,0.924824,0.178512,0.335909,-0.232285,0.220570,-6.786203
9,758,279.2379,0.88044,0.47257,12,4.37163,10.971636,19.724611,-3.402405e+07,73.109451,...,-3.399828e+07,0.310489,0.858282,0.372139,0.924120,0.179156,0.337500,-0.239111,0.222999,-6.769002


## PART 2 给出t0,算出t0的F,G,进而得出r0,v0

In [23]:
SG_t0,jd_t0 = DDS.UTC2SG(1998,12,4,0)
t0 = ( (jd_t0)*24*3600)/time_unit
position_fram_fil['deltaT'] = position_fram_fil['formatT'].values - t0
position_fram_fil

,Number,lambda,Rcos_fai,Rsin_fai,month,day,alpha,delta,SG,SG_d,...,X,Y,Z,Lambda,Miu,Niu,P,Q,formatT,deltaT
0,97,34.7625,0.86165,0.50608,12,4.13532,10.960203,19.700000,-3.402409e+07,72.961353,...,0.310448,0.858320,0.372139,0.924298,0.178999,0.337095,-0.239317,0.222723,-6.769009,0.000004
1,97,34.7625,0.86165,0.50608,12,4.14534,10.960672,19.700000,-3.402409e+07,72.967632,...,0.310447,0.858320,0.372139,0.924296,0.179007,0.337095,-0.239317,0.222720,-6.769009,0.000004
2,97,34.7625,0.86165,0.50608,12,5.09543,11.006331,19.783333,-3.393746e+07,73.922000,...,0.293800,0.863099,0.374211,0.923671,0.179649,0.338464,-0.246207,0.224901,-6.751808,0.017205
3,97,34.7625,0.86165,0.50608,12,5.09868,11.006483,19.783333,-3.393746e+07,73.924037,...,0.293800,0.863099,0.374211,0.923671,0.179652,0.338464,-0.246207,0.224900,-6.751808,0.017205
4,758,279.2379,0.88044,0.47257,12,3.36964,10.923786,19.625083,-3.411069e+07,72.122556,...,0.327042,0.853240,0.369953,0.924843,0.178495,0.335864,-0.232307,0.220538,-6.786204,-0.017191
5,758,279.2379,0.88044,0.47257,12,3.37277,10.923942,19.625583,-3.411069e+07,72.124518,...,0.327041,0.853240,0.369953,0.924840,0.178497,0.335872,-0.232303,0.220544,-6.786204,-0.017191
6,758,279.2379,0.88044,0.47257,12,3.38338,10.924431,19.726667,-3.411069e+07,72.131167,...,0.327041,0.853240,0.369953,0.924255,0.178392,0.337533,-0.231544,0.222000,-6.786204,-0.017191
7,758,279.2379,0.88044,0.47257,12,3.38649,10.924575,19.627000,-3.411069e+07,72.133116,...,0.327041,0.853240,0.369953,0.924830,0.178506,0.335895,-0.232292,0.220561,-6.786204,-0.017191
8,758,279.2379,0.88044,0.47257,12,3.39707,10.925042,19.627806,-3.411069e+07,72.139747,...,0.327041,0.853240,0.369953,0.924824,0.178512,0.335909,-0.232285,0.220570,-6.786203,-0.017190
9,758,279.2379,0.88044,0.47257,12,4.37163,10.971636,19.724611,-3.402405e+07,73.109451,...,0.310489,0.858282,0.372139,0.924120,0.179156,0.337500,-0.239111,0.222999,-6.769002,0.000011


In [17]:
observed_choose=list(np.arange(len(position_fram_fil)))
# observed_choose = [4,5,6]
r0_a,v0_a = Calculatefort0(position_fram_fil,observed_choose,t0,1e-8)


0 : 4.779865221398749 0.8386836763650766
[0.27122356 0.27122361 0.27355597 0.27355598 0.26890907 0.26890908
 0.26890912 0.26890913 0.26890918 0.27122452 0.27122456 0.27122457
 0.27122458 0.27122459 0.27122315 0.27122316 0.27122316 0.27122317
 0.27122485 0.2712249  0.27122495 0.27122501 0.25099071 0.25099075
 0.25099079 0.25099083 0.26433115 0.2643312  0.26433124 0.26661239
 0.26661241 0.26661242 0.26661275 0.26661278 0.26661282 0.27122623
 0.27122625 0.27122626 0.27122661 0.27122662 0.27122664] [0.91139204 0.91139224 0.92276849 0.92276855 0.90013785 0.90013791
 0.90013812 0.90013818 0.90013838 0.91139668 0.91139687 0.91139693
 0.91139699 0.91139705 0.91139004 0.91139007 0.91139009 0.91139012
 0.91139828 0.91139853 0.91139878 0.91139907 0.81421908 0.81421927
 0.81421946 0.81421964 0.87798145 0.87798165 0.87798186 0.889005
 0.88900509 0.88900517 0.88900672 0.88900689 0.88900706 0.91140502
 0.9114051  0.91140518 0.91140686 0.91140693 0.91140701]
--------------------
1 : 4.500169105147998 

In [19]:
r0 = np.sqrt(np.dot(r0_a,r0_a))
v0 = np.sqrt(np.dot(v0_a,v0_a))

print(r0_a*au,'r0:',r0*au,'km = ',r0,'au;')

print(v0_a*au/time_unit,'v0:',v0*au/time_unit,'km/s =',v0,'au/[T];')

print(t0*time_unit,'s')

[-2.14489673e+08 -4.37327882e+08 -4.63669737e+08] r0: 672496186.6644936 km =  4.495359370186845 au;
[ 6.67382965 14.95469166 13.09341292] v0: 20.967123473539655 km/s = 0.7039563712836449 au/[T];
-33998335.815999985 s


In [20]:
v0**2 * r0

2.2276958916947844

# 已知某一时刻的位置矢量和速度矢量，求轨道六根数

按照笔记方法,逐步求得：
* a
* n()
* E()
* e
* M
* i Omega w

In [21]:
# 从第二步开始，因为不需要用到F,G的计算，所以不用再取理论单位，现在取国际单位制
# astronomy constants
R_earth = 6371e3 # m
au = 149597870e3 # m

M_earth = 5.965e24 # kg
M_sun = 1.989e30 # kg

G_graviation = 6.672e-11 # N·m^2 /kg^2 


r_station_earth = 0.999102 #*R_earth
r_earth_sun = 1*au

### unit for time -> artifact satellite
# time_unit = 806.81163 #806.8116 # SI 

## unit for time -> planet
time_unit_day = 58.1324409 # Mean solar day
time_unit = time_unit_day * 86400

# miu_GM = 398600.5e-6 # km^3/SI^2

miu_GM = G_graviation*M_sun # m^3/SI^2

In [22]:
a,e,M,i,Omega,w,P,Q,R = Calculatefor6elements(r0_a,v0_a,t0,0)
print(a/au,'au') 

a -2952100291484.007 m
n: nan rad/s - Not Output
e: nan Nan
E: nan rad - Not Output
M: nan rad
i,Omega,w: 1.799517164929409 1.3382677153777423 nan rad/rad/rad
-19.733571684436463 au


/Users/dengdingshan/Documents/GitHub/dds_python/Celestial mechanics/DDS_Laplace.py:203: RuntimeWarning: invalid value encountered in sqrt
  n = np.sqrt(miu_GM/(a**3))


In [96]:
np.degrees(np.array([M,i,Omega,w]))

array([357.13326033,  22.56377063, 351.16077836, -83.3641624 ])

In [97]:
P,Q

(array([-0.09495879,  0.92299792,  0.37290437]),
 array([-0.95447112,  0.04276741, -0.04338289]))

#  由轨道6根数归算星历表

已知：
* a, # m
* n(), # rad/s
* E(), # rad
* e, # Nan
* M, # rad
* [i, Omega, w] = P,Q # [rad,rad,rad]

t0 **# unit = SI from the first data we have**

M = M0 + n(t-t0)

M = E - esin(E)

In [98]:
print('a',a,'e',e,'M',M,i,Omega,w)

a 496106227721.58374 e 0.7133498941682178 M 6.233151261169933 0.39381208923640454 6.128911786260771 -1.454979111967492


In [99]:
# 假设我们已知a,e,M,i,Omega,w,那么E,n均需要从六根数推出来。

n = np.sqrt(miu_GM/a**3)

n

3.29673009050317e-08

In [100]:
# calculate keplar equation
# 设置初值t0 和我们需要归算的时间t

# 我将计算以整个周期为倍数的时间序列
# set T

T = 2*np.pi/n
print(T)


190588405.31954506


In [101]:
# set t

# t = np.arange(-201,T,) # unit = SI

t = (position_fram_fil['formatT'].values - t0)*time_unit

t_forecast = t
Mt = (M + n*(t))%(2*np.pi)

print(t/24/3600)
print(Mt)

[ 2.34930555e-04  2.52326389e-04  1.00016568e+00  1.00017132e+00
 -9.99358264e-01 -9.99352830e-01 -9.99334410e-01 -9.99329010e-01
 -9.99310642e-01  6.45190973e-04  6.61545139e-04  6.66892362e-04
  6.72343750e-04  6.77812500e-04  5.84722228e-05  6.08680549e-05
  6.26736108e-05  6.50173604e-05  7.86111111e-04  8.08524306e-04
  8.30607639e-04  8.55781250e-04 -8.99920826e+00 -8.99919000e+00
 -8.99917161e+00 -8.99915326e+00 -2.99848259e+00 -2.99846429e+00
 -2.99844488e+00 -1.99859769e+00 -1.99858995e+00 -1.99858266e+00
 -1.99844245e+00 -1.99842724e+00 -1.99841184e+00  1.38199653e-03
  1.38868056e-03  1.39548611e-03  1.54407986e-03  1.55090278e-03
  1.55788194e-03]
[6.23315193 6.23315198 6.23600011 6.23600012 6.23030471 6.23030473
 6.23030478 6.2303048  6.23030485 6.2331531  6.23315315 6.23315316
 6.23315318 6.23315319 6.23315143 6.23315143 6.23315144 6.23315145
 6.2331535  6.23315356 6.23315363 6.2331537  6.20751814 6.2075182
 6.20751825 6.2075183  6.22461046 6.22461051 6.22461057 6.2274585

In [102]:
# 迭代 开普勒方程解 E
Et = []
epsilon = 1e-6
for M_single in Mt:
#     print(M_single)
    key = 0
    En0 = M_single
    Enw = [En0]
    Nwind = 0
    while key == 0:
        Enw.append(M_single + e*np.sin(Enw[Nwind]))

        Nwind += 1
#         print(Enw[Nwind])

        delta_Enw = np.abs(Enw[Nwind] - Enw[Nwind - 1])
#         print(delta_Enw)
#         print('------------')

        if delta_Enw < epsilon:
            key = 1;

    E_single = Enw[Nwind]
    print(E_single)
    Et.append(E_single)
# print('E:',Et)

6.110765358010259
6.1107655247167765
6.120366446036631
6.120366500316709
6.101205282336561
6.1012053342018575
6.101205510013579
6.101205561547471
6.101205736862109
6.110769289591799
6.110769446316138
6.110769497559346
6.110769549800791
6.110769602208614
6.110763666987948
6.110763689947498
6.110763707250362
6.110763729710799
6.11077064005017
6.1107708548391795
6.110771066467091
6.1107713077096175
6.026230463207131
6.026230630984205
6.026230799877693
6.026230968452226
6.082205309112745
6.082205482268347
6.082205665938153
6.0916879273851805
6.091688000978337
6.09168807028131
6.091689402878899
6.091689547425195
6.09168969378658
6.110776350515907
6.110776414570096
6.110776479788914
6.110777903788862
6.110777969174095
6.110778036056688


In [103]:
# 给出P_new, Q_new
P_new = np.array([np.cos(Omega)*np.cos(w) - np.sin(Omega)*np.sin(w)*np.cos(i),\
                 np.sin(Omega)*np.cos(w) + np.cos(Omega)*np.sin(w)*np.cos(i),\
                 np.sin(w)*np.sin(i) ])
Q_new = np.array([-np.cos(Omega)*np.sin(w) - np.sin(Omega)*np.cos(w)*np.cos(i),\
                 -np.sin(Omega)*np.sin(w) + np.cos(Omega)*np.cos(w)*np.cos(i),\
                 np.cos(w)*np.sin(i)])

print(P,Q)
print(P_new,Q_new)

[-0.09495879  0.92299792  0.37290437] [-0.95447112  0.04276741 -0.04338289]
[-0.02676325 -0.92412952 -0.38114087] [ 0.9979014  -0.04718742  0.04434111]


# Warnning: P，Q反算出来的值不对

In [58]:
# 归算rt, vt
rt = [];vt=[];
for Et_single in Et:
    rt_single = a*(np.cos(Et_single) - e)*P_new + a*np.sqrt(1 - e**2)*np.sin(Et_single)*Q_new
    
    rt_single_norm = np.sqrt( np.dot(rt_single,rt_single) )
    rt.append(rt_single)
    
    vt_single = -a**2*n/rt_single_norm*np.sin(Et_single)*P_new \
    + a**2*n/rt_single_norm*np.sqrt(1 - e**2)*np.cos(Et_single)*Q_new
    
    vt_single_norm = np.sqrt( np.dot(vt_single,vt_single) )
    vt.append(rt_single)
    
#     print(rt_single_norm/10**3,vt_single_norm/10**3)
    
rt = np.array(rt)
vt = np.array(vt)

In [26]:
# 计算预测的方向矢量L，赤经，赤纬
P_earth = rt
station_earth = np.array([-0.81000791, -0.25188507,  0.52786921])*R_earth

P_station = P_earth - station_earth

L_forecast_total = [];
Alpha_forecast = [];
Delta_forecast = [];
Nwind = 0
for P_station_single in P_station:
    # 计算我们测站的赤经Alpha_forecast 和赤纬Delta_forecast
    L_forecast_total.append( P_station_single/np.sqrt(np.dot(P_station_single,P_station_single)) )
    
    Delta_forecast.append( np.arcsin(L_forecast_total[Nwind][2]) )
    Alpha_forecast.append( arctan2(L_forecast_total[Nwind][1]/np.cos(Delta_forecast[Nwind]),\
                                   L_forecast_total[Nwind][0]/np.cos(Delta_forecast[Nwind])) )
    
    Nwind += 1;

##############################################

Forecast_fram = pd.DataFrame({'Alpha': np.array(Alpha_forecast),
                              'Alpha_d':np.degrees(np.array(Alpha_forecast)),
                             'Delta':np.array(Delta_forecast),
                              'Delta_d':np.degrees(np.array(Delta_forecast)),
                             'deltaT':t_forecast-t_forecast[0],
                             'x':rt[:,0],
                             'y':rt[:,1],
                             'z':rt[:,2],
                             'vx':vt[:,0],
                             'vy':vt[:,1],
                             'vz':vt[:,2]})

# print(L_forecast_total)

In [27]:
Forecast_fram.to_csv('echo1st_forecast.csv')
Forecast_fram


,Alpha,Alpha_d,Delta,Delta_d,deltaT,vx,vy,vz,x,y,z
0,2.498373,143.146252,0.151886,8.702443,0.000000,-6.961152e+06,-2.551088e+05,3.707492e+06,-6.961152e+06,-2.551088e+05,3.707492e+06
1,2.752704,157.718321,-0.036782,-2.107448,100.735981,-7.115216e+06,-8.038272e+05,3.285322e+06,-7.115216e+06,-8.038272e+05,3.285322e+06
2,3.019047,172.978645,-0.252186,-14.449184,202.649000,-7.211310e+06,-1.352183e+06,2.830642e+06,-7.211310e+06,-1.352183e+06,2.830642e+06
3,3.230637,185.101892,-0.414670,-23.758859,285.706998,-7.244362e+06,-1.790804e+06,2.442127e+06,-7.244362e+06,-1.790804e+06,2.442127e+06
4,3.710786,212.612386,-0.701128,-40.171673,500.276999,-7.139355e+06,-2.870855e+06,1.379838e+06,-7.139355e+06,-2.870855e+06,1.379838e+06
5,3.904895,223.733979,-0.782428,-44.829838,603.889999,-6.990509e+06,-3.355575e+06,8.454528e+05,-6.990509e+06,-3.355575e+06,8.454528e+05


In [28]:
position_fram_fil

,UT1_hours,alpha,delta,year,month,days,deltaT,Lambda,Miu,Niu,P,Q,SG_d,deltaT_SI,jd
0,21.575128,142.935000,8.521111,1965,1,14,0.000000,-0.789144,0.596067,0.148174,0.296272,-0.351086,78.032128,0.000000,-12769.600293
1,21.603111,157.274167,-2.395000,1965,1,14,0.124857,-0.921558,0.385984,-0.041788,0.520311,-0.193223,78.453011,100.735981,-12769.599128
2,21.631420,171.817917,-14.508333,1965,1,14,0.251173,-0.958257,0.137781,-0.250521,0.708283,-0.008122,78.878811,202.649000,-12769.597948
3,21.654491,183.134167,-23.563333,1965,1,14,0.354119,-0.915248,-0.050115,-0.399763,0.805554,0.131507,79.225833,285.706998,-12769.596987
4,21.714094,208.640417,-40.020278,1965,1,14,0.620067,-0.672115,-0.367065,-0.643059,0.870732,0.370844,80.122323,500.276999,-12769.594503
5,21.742876,219.102083,-44.998611,1965,1,14,0.748489,-0.548745,-0.445986,-0.707090,0.855495,0.434417,80.555226,603.889999,-12769.593304


# 计算残差 见 step3 - echo1st (在小行星部分不计算）